In [2]:
import pandas as pd
import numpy as np
import math
import pickle

from scipy import stats
import scipy.io
from scipy.spatial.distance import pdist
from scipy.linalg import cholesky
from scipy.io import loadmat

import matlab.engine as engi
import matlab as mat

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,roc_auc_score,recall_score,precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from pyearth import Earth
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

from src import SMOTE
from src import CFS
from src import metrices_V2 as metrices

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path

import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/suvodeepmajumder/Conda/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
def apply_smote(df):
    df_buggy = df[df['Bugs'] == True]
    df_non_buggy = df[df['Bugs'] == False]
    if df_buggy.shape[0] <= 10:
        for i in range(5):
            df = pd.concat([df,df_buggy], axis = 0)
    if df_non_buggy.shape[0] <= 10:
        for i in range(5):
            df = pd.concat([df,df_non_buggy], axis = 0)
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = SMOTE.smote(df)
    df = smt.run()
    df.columns = cols
    return df

def apply_cfs(df):
        y = df.Bugs.values
        X = df.drop(labels = ['Bugs'],axis = 1)
        X = X.values
        selected_cols = CFS.cfs(X,y)
        cols = df.columns[[selected_cols]].tolist()
        cols.append('Bugs')
        return df[cols],cols

In [4]:
def load_product_data(project):
    # Processing Product files
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    df = understand_df
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    _df = df
    df = df[cols]
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y,_df

In [5]:
def load_process_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    commits = understand_df.commit_hash.unique()
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_path = 'data/commit_guru/' + project + '.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_df = pd.read_csv(commit_guru_path)
    commit_guru_df = commit_guru_df[['commit_hash','contains_bug']]
    values = {'contains_bug': False}
    commit_guru_df = commit_guru_df.fillna(values)
#     commit_guru_df = commit_guru_df[commit_guru_df.commit_hash.isin(commits)]
    df = commit_guru_file_level_df.merge(commit_guru_df,on='commit_hash') 
    df.rename(columns={"contains_bug": "Bugs"},inplace=True)
    df = df[df['file_name'].str.contains('.java')]
    df = df.drop(['commit_hash','file_name'],axis = 1)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.drop_duplicates()
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    return X,y

In [11]:
def load_both_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
    commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    
    
    df = understand_df.merge(commit_guru_file_level_df,how='left',on=['commit_hash','Name'])
    
    
    cols = df.columns.tolist()
    cols.remove('Bugs')
    cols.append('Bugs')
    df = df[cols]
    
    for item in ['Kind', 'Name','commit_hash']:
        if item in cols:
            df = df.drop(labels = [item],axis=1)
#     df.dropna(inplace=True)
    df = df.drop_duplicates()
    df.reset_index(drop=True, inplace=True)
    
    y = df.Bugs
    X = df.drop('Bugs',axis = 1)
    cols = X.columns
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    imp_mean = IterativeImputer(random_state=0)
    X = imp_mean.fit_transform(X)
    X = pd.DataFrame(X,columns = cols)
    
#     X = X[['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd']]

#     X = X.drop(['la', 'ld', 'lt', 'age', 'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd'],axis = 1)
    
    return X,y

In [7]:
def load_more_data(project):
    understand_path = 'data/understand_files_all/' + project + '_understand.csv'
    understand_df = pd.read_csv(understand_path)
    understand_df = understand_df.dropna(axis = 1,how='all')
    cols_list = understand_df.columns.values.tolist()
    for item in ['Kind', 'Name','commit_hash', 'Bugs']:
        if item in cols_list:
            cols_list.remove(item)
            cols_list.insert(0,item)
    understand_df = understand_df[cols_list]
    cols = understand_df.columns.tolist()
    understand_df = understand_df.drop_duplicates(cols[4:len(cols)])
    understand_df['Name'] = understand_df.Name.str.rsplit('.',1).str[1]
    u_files = understand_df.Name.unique()
    
    commit_guru_file_level_path = 'data/commit_guru_file_level/' + project + '_file.csv'
    commit_guru_file_level_df = pd.read_csv(commit_guru_file_level_path)
    commit_guru_file_level_df['commit_hash'] = commit_guru_file_level_df.commit_hash.str.strip('"')
    commit_guru_file_level_df = commit_guru_file_level_df[commit_guru_file_level_df['file_name'].str.contains('.java')]
    commit_guru_file_level_df['Name'] = commit_guru_file_level_df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')
#     commit_guru_file_level_df = commit_guru_file_level_df.drop('file_name',axis = 1)
    commit_guru_file_level_df.reset_index(inplace=True,drop=True)
    
    u_commits = commit_guru_file_level_df.commit_hash.unique()
    
    final_df = pd.DataFrame()
    for commit in u_commits:
        commit_df = commit_guru_file_level_df[commit_guru_file_level_df['commit_hash'] == commit]
        cols = commit_df.columns
        start_index = commit_df.index[0]
        if start_index == 0:
            continue
        sub_df = commit_guru_file_level_df.loc[:start_index]
        metrics = []
        for file in u_files:
            if file not in list(commit_df.Name.unique()):
                sub_df_file = sub_df[sub_df['Name'] == file]
                sub_df_file = sub_df_file.reset_index(drop = True)
                if sub_df_file.shape[0] > 0:
    #                 print(sub_df_file)
                    metrics.append(sub_df_file.loc[sub_df_file.shape[0]-1].values.tolist()[1:])
        metrics_df = pd.DataFrame(metrics,columns = cols[1:])
        metrics_df['commit_hash'] = [commit]*metrics_df.shape[0]
        metrics_df = metrics_df[['commit_hash','Name','file_name' ,'la', 'ld', 'lt', 'age', 
                              'ndev', 'nuc', 'ns', 'exp', 'sexp', 'rexp', 'nd']]
        metrics_df = pd.concat([metrics_df,commit_df],axis = 0)
        final_df = pd.concat([final_df,metrics_df],axis = 0)
    return final_df


In [8]:
def run_self(project):
    X,y = load_both_data(project)
    print(X.shape,y.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
    loc = X_test.CountLineCode
#     loc = X_test['la'] + X_test['lt']
    df_smote = pd.concat([X_train,y_train],axis = 1)
    df_smote = apply_smote(df_smote)
    y_train = df_smote.Bugs
    X_train = df_smote.drop('Bugs',axis = 1)
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    importance = clf.feature_importances_
    print(len(importance))
    predicted = clf.predict(X_test)
    abcd = metrices.measures(y_test,predicted,loc)
    pf = abcd.get_pf()
    recall = abcd.calculate_recall()
    precision = abcd.calculate_precision()
    f1 = abcd.calculate_f1_score()
    g_score = abcd.get_g_score()
    pci_20 = abcd.get_pci_20()
    ifa = abcd.get_ifa()
    try:
        auc = roc_auc_score(y_test, predicted)
    except:
        auc = 0
    print(classification_report(y_test, predicted))
    return recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance

In [8]:
proj_df = pd.read_csv('projects.csv')
projects = proj_df.repo_name.tolist()

In [9]:
precision_list = {}
recall_list = {}
pf_list = {}
f1_list = {}
g_list = {}
auc_list = {}
pci_20_list = {}
ifa_list = {}
featue_importance = {}
for project in projects[150:]:
    try:
        if project == '.DS_Store':
            continue
#         if project != 'org.alloytools.alloy':
#             continue
#         if project != 'android-simple-facebook':
#             continue
        print("+++++++++++++++++   "  + project + "  +++++++++++++++++")
        recall,precision,pf,f1,g_score,auc,pci_20,ifa,importance = run_self(project)
        recall_list[project] = recall
        precision_list[project] = precision
        pf_list[project] = pf
        f1_list[project] = f1
        g_list[project] = g_score
        auc_list[project] = auc
        pci_20_list[project] = pci_20
        ifa_list[project] = ifa
        featue_importance[project] = importance
    except Exception as e:
        print(e)
        continue
final_result = {}
final_result['precision'] = precision_list
final_result['recall'] = recall_list
final_result['pf'] = pf_list
final_result['f1'] = f1_list
final_result['g'] = g_list
final_result['auc'] = auc_list
final_result['pci_20'] = pci_20_list
final_result['ifa'] = ifa_list
final_result['featue_importance'] = featue_importance

+++++++++++++++++   spring-xd-samples  +++++++++++++++++
(66, 45) (66,)
45
              precision    recall  f1-score   support

           0       0.75      0.56      0.64        16
           1       0.53      0.73      0.62        11

    accuracy                           0.63        27
   macro avg       0.64      0.64      0.63        27
weighted avg       0.66      0.63      0.63        27

+++++++++++++++++   environment-dashboard  +++++++++++++++++
(46, 45) (46,)
45
              precision    recall  f1-score   support

           0       0.53      1.00      0.69         9
           1       1.00      0.20      0.33        10

    accuracy                           0.58        19
   macro avg       0.76      0.60      0.51        19
weighted avg       0.78      0.58      0.50        19

+++++++++++++++++   plantuml  +++++++++++++++++
(7307, 45) (7307,)
45
              precision    recall  f1-score   support

           0       0.70      0.78      0.74      1936
           1 

(330, 45) (330,)
45
              precision    recall  f1-score   support

           0       0.45      0.38      0.41        37
           1       0.77      0.82      0.80        95

    accuracy                           0.70       132
   macro avg       0.61      0.60      0.60       132
weighted avg       0.68      0.70      0.69       132

+++++++++++++++++   react-native-splash-screen  +++++++++++++++++
(33, 45) (33,)
45
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       1.00      0.50      0.67         6

    accuracy                           0.79        14
   macro avg       0.86      0.75      0.75        14
weighted avg       0.84      0.79      0.77        14

+++++++++++++++++   react-native-incall-manager  +++++++++++++++++
(44, 45) (44,)
45
              precision    recall  f1-score   support

           0       1.00      0.86      0.92        14
           1       0.67      1.00      0.80     

(150, 45) (150,)
45
              precision    recall  f1-score   support

           0       0.84      0.67      0.74        48
           1       0.27      0.50      0.35        12

    accuracy                           0.63        60
   macro avg       0.56      0.58      0.55        60
weighted avg       0.73      0.63      0.67        60

+++++++++++++++++   community  +++++++++++++++++
Shape of passed values is (62, 45), indices imply (62, 56)
+++++++++++++++++   android-floating-action-button  +++++++++++++++++
(60, 45) (60,)
45
              precision    recall  f1-score   support

           0       0.75      0.83      0.79        18
           1       0.25      0.17      0.20         6

    accuracy                           0.67        24
   macro avg       0.50      0.50      0.49        24
weighted avg       0.62      0.67      0.64        24

+++++++++++++++++   RxCache  +++++++++++++++++
(385, 45) (385,)
45
              precision    recall  f1-score   support

        

(283, 45) (283,)
45
              precision    recall  f1-score   support

           0       0.80      0.69      0.74        83
           1       0.40      0.55      0.46        31

    accuracy                           0.65       114
   macro avg       0.60      0.62      0.60       114
weighted avg       0.69      0.65      0.66       114

+++++++++++++++++   EasyNavigation  +++++++++++++++++
(88, 45) (88,)
45
              precision    recall  f1-score   support

           0       0.88      0.82      0.85        17
           1       0.85      0.89      0.87        19

    accuracy                           0.86        36
   macro avg       0.86      0.86      0.86        36
weighted avg       0.86      0.86      0.86        36

+++++++++++++++++   RxFingerprint  +++++++++++++++++
(141, 45) (141,)
45
              precision    recall  f1-score   support

           0       0.65      0.57      0.61        35
           1       0.42      0.50      0.46        22

    accuracy     

(100, 45) (100,)
45
              precision    recall  f1-score   support

           0       0.55      0.50      0.52        22
           1       0.45      0.50      0.47        18

    accuracy                           0.50        40
   macro avg       0.50      0.50      0.50        40
weighted avg       0.51      0.50      0.50        40

+++++++++++++++++   Android-ReactiveLocation  +++++++++++++++++
(185, 45) (185,)
45
              precision    recall  f1-score   support

           0       0.67      0.62      0.64        45
           1       0.47      0.52      0.49        29

    accuracy                           0.58        74
   macro avg       0.57      0.57      0.57        74
weighted avg       0.59      0.58      0.58        74

+++++++++++++++++   gradle-retrolambda  +++++++++++++++++
(60, 45) (60,)
45
              precision    recall  f1-score   support

           0       0.17      0.17      0.17         6
           1       0.72      0.72      0.72        18

  

...

(1084, 45) (1084,)
45
              precision    recall  f1-score   support

           0       0.82      0.90      0.85       339
           1       0.43      0.27      0.33        95

    accuracy                           0.76       434
   macro avg       0.62      0.59      0.59       434
weighted avg       0.73      0.76      0.74       434

+++++++++++++++++   GCViewer  +++++++++++++++++
(1835, 45) (1835,)
45
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       405
           1       0.57      0.57      0.57       329

    accuracy                           0.62       734
   macro avg       0.61      0.61      0.61       734
weighted avg       0.62      0.62      0.62       734

+++++++++++++++++   lucida  +++++++++++++++++
(1110, 45) (1110,)
45
              precision    recall  f1-score   support

           0       0.89      0.93      0.91       382
           1       0.38      0.27      0.32        62

    accuracy          

Shape of passed values is (3196, 45), indices imply (3196, 56)
+++++++++++++++++   uhabits  +++++++++++++++++
(4073, 45) (4073,)
45
              precision    recall  f1-score   support

           0       0.60      0.60      0.60       727
           1       0.68      0.67      0.67       903

    accuracy                           0.64      1630
   macro avg       0.64      0.64      0.64      1630
weighted avg       0.64      0.64      0.64      1630

+++++++++++++++++   spock  +++++++++++++++++
(3274, 45) (3274,)
45
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       950
           1       0.51      0.38      0.43       360

    accuracy                           0.73      1310
   macro avg       0.65      0.62      0.63      1310
weighted avg       0.71      0.73      0.71      1310

+++++++++++++++++   PageTurner  +++++++++++++++++
(2145, 45) (2145,)
45
              precision    recall  f1-score   support

           0       0

(6213, 45) (6213,)
45
              precision    recall  f1-score   support

           0       0.68      0.69      0.68      1483
           1       0.53      0.53      0.53      1003

    accuracy                           0.62      2486
   macro avg       0.61      0.61      0.61      2486
weighted avg       0.62      0.62      0.62      2486

+++++++++++++++++   spring-social  +++++++++++++++++
(2903, 45) (2903,)
45
              precision    recall  f1-score   support

           0       0.74      0.75      0.74       814
           1       0.40      0.39      0.39       348

    accuracy                           0.64      1162
   macro avg       0.57      0.57      0.57      1162
weighted avg       0.64      0.64      0.64      1162

+++++++++++++++++   pebble  +++++++++++++++++
(3195, 45) (3195,)
45
              precision    recall  f1-score   support

           0       0.58      0.51      0.54       473
           1       0.73      0.78      0.75       805

    accuracy     

(2531, 45) (2531,)
45
              precision    recall  f1-score   support

           0       0.70      0.69      0.70       558
           1       0.63      0.64      0.64       455

    accuracy                           0.67      1013
   macro avg       0.67      0.67      0.67      1013
weighted avg       0.67      0.67      0.67      1013

+++++++++++++++++   cassandra-lucene-index  +++++++++++++++++
(5089, 45) (5089,)
45
              precision    recall  f1-score   support

           0       0.60      0.52      0.56       611
           1       0.81      0.85      0.83      1425

    accuracy                           0.75      2036
   macro avg       0.70      0.69      0.69      2036
weighted avg       0.74      0.75      0.75      2036

+++++++++++++++++   AndroidAsync  +++++++++++++++++
(3389, 45) (3389,)
45
              precision    recall  f1-score   support

           0       0.82      0.83      0.82       856
           1       0.70      0.68      0.69       500

  

(5475, 45) (5475,)
45
              precision    recall  f1-score   support

           0       0.49      0.46      0.48       971
           1       0.59      0.62      0.60      1219

    accuracy                           0.55      2190
   macro avg       0.54      0.54      0.54      2190
weighted avg       0.55      0.55      0.55      2190

+++++++++++++++++   Discord4J  +++++++++++++++++
(6014, 45) (6014,)
45
              precision    recall  f1-score   support

           0       0.54      0.46      0.50       932
           1       0.69      0.75      0.72      1474

    accuracy                           0.64      2406
   macro avg       0.62      0.61      0.61      2406
weighted avg       0.63      0.64      0.64      2406

+++++++++++++++++   OpenRefine  +++++++++++++++++
(8588, 45) (8588,)
45
              precision    recall  f1-score   support

           0       0.68      0.66      0.67      1832
           1       0.62      0.64      0.63      1604

    accuracy     

In [10]:
with open('results/Performance/commit_guru_file_specific/product_700_rf.pkl', 'wb') as handle:
    pickle.dump(final_result, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
np.nanmedian(list(auc_list.values()))

0.616413315350171

In [184]:
featue_importance_df = pd.DataFrame.from_dict(featue_importance,orient = 'index')
featue_importance_df = featue_importance_df.dropna(axis = 1)
featue_importance_df.columns = X.columns

In [185]:
featue_importance_df.median().sort_values()

ns      0.000000
nd      0.000000
nuc     0.034993
lt      0.044638
la      0.065074
ld      0.074186
sexp    0.077190
ndev    0.084676
exp     0.086341
age     0.148113
rexp    0.274957
dtype: float64

In [13]:
X.columns

Index(['AvgCyclomatic', 'AvgCyclomaticModified', 'AvgCyclomaticStrict',
       'AvgEssential', 'AvgLine', 'AvgLineBlank', 'AvgLineCode',
       'AvgLineComment', 'CountClassBase', 'CountClassCoupled',
       'CountClassCoupledModified', 'CountClassDerived',
       'CountDeclClassMethod', 'CountDeclClassVariable',
       'CountDeclInstanceMethod', 'CountDeclInstanceVariable',
       'CountDeclMethod', 'CountDeclMethodAll', 'CountDeclMethodDefault',
       'CountDeclMethodPrivate', 'CountDeclMethodProtected',
       'CountDeclMethodPublic', 'CountLine', 'CountLineBlank', 'CountLineCode',
       'CountLineCodeDecl', 'CountLineCodeExe', 'CountLineComment',
       'CountSemicolon', 'CountStmt', 'CountStmtDecl', 'CountStmtExe',
       'MaxCyclomatic', 'MaxCyclomaticModified', 'MaxCyclomaticStrict',
       'MaxEssential', 'MaxInheritanceTree', 'MaxNesting',
       'PercentLackOfCohesion', 'PercentLackOfCohesionModified',
       'RatioCommentToCode', 'SumCyclomatic', 'SumCyclomaticModified',
 

In [12]:
X,y = load_both_data('nokogiri')

In [117]:
_,_,df_pr = load_product_data('org.alloytools.alloy')
# df_pr['Name'] = df_pr.Name.str.rsplit('.',1).str[1]
df_pr

,Bugs,commit_hash,Name,Kind,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,...,MaxEssential,MaxInheritanceTree,MaxNesting,PercentLackOfCohesion,PercentLackOfCohesionModified,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
0,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.ApplicationTest,Public Class,1.0,1.0,1.0,1.0,3.0,0.0,...,1.0,2.0,0,0.0,0.0,0.75,1,1,1,1
1,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MANAndroidDemo,Public Class,3.0,3.0,3.0,1.0,60.0,4.0,...,1.0,1.0,3,0.0,0.0,0.89,6,6,6,2
2,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MainActivity,Public Class,1.0,1.0,1.0,1.0,17.0,1.0,...,1.0,2.0,1,0.0,0.0,0.11,4,4,4,3
3,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.AboutUsActivity,Public Class,1.0,1.0,1.0,1.0,6.0,0.0,...,1.0,2.0,1,0.0,0.0,0.17,4,4,4,3
6,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.HttpDnsTLSSniSock...,Public Class,1.0,1.0,1.0,1.0,7.0,0.0,...,1.0,2.0,2,85.0,85.0,0.13,14,14,14,9
7,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity,Public Class,2.0,1.0,2.0,1.0,20.0,0.0,...,1.0,2.0,2,89.0,78.0,0.24,36,29,38,16
8,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.http...,Anonymous Class,4.0,4.0,4.0,1.0,53.0,1.0,...,1.0,2.0,1,0.0,0.0,0.35,4,4,4,1
9,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.http...,Anonymous Class,2.0,2.0,2.0,1.0,7.0,0.0,...,1.0,2.0,1,0.0,0.0,1.22,2,2,2,1
10,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.init...,Anonymous Class,2.0,2.0,2.0,1.0,7.0,0.0,...,1.0,2.0,1,0.0,0.0,0.00,2,2,2,1
11,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.norm...,Anonymous Class,4.0,4.0,4.0,1.0,34.0,3.0,...,1.0,2.0,2,0.0,0.0,0.14,4,4,4,1


In [115]:
new_df = df_pr.merge(df,on=['commit_hash','Name'])
new_df = new_df.drop(['commit_hash','Name','Kind'],axis = 1)
# new_df = new_df.dropna()
new_df.shape

(0, 58)

In [30]:
y = new_df.Bugs
X = new_df.drop('Bugs',axis = 1)
cols = X.columns
imp_mean = IterativeImputer(random_state=0)
X = imp_mean.fit_transform(X)
X = pd.DataFrame(X,columns = cols)

In [31]:
# X = X.drop(['AvgCyclomatic', 'AvgCyclomaticModified', 'AvgCyclomaticStrict',
#        'AvgEssential', 'AvgLine', 'AvgLineBlank', 'AvgLineCode',
#        'AvgLineComment', 'CountClassBase', 'CountClassCoupled',
#        'CountClassCoupledModified', 'CountClassDerived',
#        'CountDeclClassMethod', 'CountDeclClassVariable',
#        'CountDeclInstanceMethod', 'CountDeclInstanceVariable',
#        'CountDeclMethod', 'CountDeclMethodAll', 'CountDeclMethodDefault',
#        'CountDeclMethodPrivate', 'CountDeclMethodProtected',
#        'CountDeclMethodPublic', 'CountLine', 'CountLineBlank', 'CountLineCode',
#        'CountLineCodeDecl', 'CountLineCodeExe', 'CountLineComment',
#        'CountSemicolon', 'CountStmt', 'CountStmtDecl', 'CountStmtExe',
#        'MaxCyclomatic', 'MaxCyclomaticModified', 'MaxCyclomaticStrict',
#        'MaxEssential', 'MaxInheritanceTree', 'MaxNesting',
#        'PercentLackOfCohesion', 'PercentLackOfCohesionModified',
#        'RatioCommentToCode', 'SumCyclomatic', 'SumCyclomaticModified',
#        'SumCyclomaticStrict', 'SumEssential'],axis = 1)
# X = X.drop(['age', 'exp', 'la', 'ld', 'lt', 'nd', 'ndev', 'ns', 'nuc', 'rexp',
#        'sexp'],axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=18)
#     loc = X_test['la'] + X_test['lt']
df_smote = pd.concat([X_train,y_train],axis = 1)
df_smote = apply_smote(df_smote)
y_train = df_smote.Bugs
X_train = df_smote.drop('Bugs',axis = 1)
clf = RandomForestClassifier()
clf.fit(X_train,y_train)
predicted = clf.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87        32
           1       0.73      0.57      0.64        14

    accuracy                           0.80        46
   macro avg       0.78      0.74      0.75        46
weighted avg       0.80      0.80      0.80        46



In [ ]:
df['Name'] = df.file_name.str.rsplit('/',1).str[1].str.split('.').str[0].str.replace('/','.')



In [61]:
df = df.reset_index(drop = True)
df[df['commit_hash'] == '1e3e80823c5ed44d55c81a597e51bc556f348a74'].file_name.values.tolist()

['oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/GetAndUploadDataDemo.java',
 'oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/GetAndUploadFileDemo.java',
 'oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/HandleException.java',
 'oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/ListObjectsInBucketDemo.java',
 'oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/MultipartUploadDemo.java',
 'oss_android_demo/src/main/java/com/alibaba/sdk/android/dpa/oss_android_demo/OSSAndroidDemo.java',
 'man_android_demo/src/androidTest/java/alibaba/man_android_demo/ApplicationTest.java',
 'man_android_demo/src/main/java/alibaba/man_android_demo/MANAndroidDemo.java',
 'man_android_demo/src/main/java/alibaba/man_android_demo/MainActivity.java',
 'man_restful_demo/src/androidTest/java/alibaba/man_restful_demo/ApplicationTest.java',
 'man_restful_demo/src/main/java/alibab

In [104]:
df['Name'] = df.file_name.str.replace('/','.').str.rsplit('.',1).str[0]

In [105]:
df_new = df.drop('file_name',axis = 1)

In [108]:
df_pr[df_pr['commit_hash'] == '1e3e80823c5ed44d55c81a597e51bc556f348a74'].Name.values

array(['alibaba.man_android_demo.ApplicationTest',
       'alibaba.man_android_demo.MANAndroidDemo',
       'alibaba.man_android_demo.MainActivity',
       'alibaba.httpdns_android_demo.AboutUsActivity',
       'alibaba.httpdns_android_demo.HttpDnsTLSSniSocketFactory',
       'alibaba.httpdns_android_demo.MainActivity',
       'alibaba.httpdns_android_demo.MainActivity.httpsRequest.(Anon_3)',
       'alibaba.httpdns_android_demo.MainActivity.httpsRequest.(Anon_3).run.(Anon_4)',
       'alibaba.httpdns_android_demo.MainActivity.initHandler.(Anon_1)',
       'alibaba.httpdns_android_demo.MainActivity.normalReqeust.(Anon_2)',
       'alibaba.httpdns_android_demo.MainActivity.setDegrationFilter.(Anon_7)',
       'alibaba.httpdns_android_demo.MainActivity.sniRequest.(Anon_5)',
       'alibaba.httpdns_android_demo.WebviewActivity',
       'alibaba.httpdns_android_demo.WebviewActivity.WebviewTlsSniSocketFactory',
       'alibaba.httpdns_android_demo.WebviewActivity.onCreate.(Anon_1)',
       

In [109]:
df_new[df_new['commit_hash'] == '1e3e80823c5ed44d55c81a597e51bc556f348a74'].Name.values

array(['oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.GetAndUploadDataDemo',
       'oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.GetAndUploadFileDemo',
       'oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.HandleException',
       'oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.ListObjectsInBucketDemo',
       'oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.MultipartUploadDemo',
       'oss_android_demo.src.main.java.com.alibaba.sdk.android.dpa.oss_android_demo.OSSAndroidDemo',
       'man_android_demo.src.androidTest.java.alibaba.man_android_demo.ApplicationTest',
       'man_android_demo.src.main.java.alibaba.man_android_demo.MANAndroidDemo',
       'man_android_demo.src.main.java.alibaba.man_android_demo.MainActivity',
       'man_restful_demo.src.androidTest.java.alibaba.man_restful_demo.ApplicationTest',
       'man_restful_demo.src.

In [111]:
df_pr[(df_pr['commit_hash'] == '1e3e80823c5ed44d55c81a597e51bc556f348a74') & (df_pr['Kind'] != 'Anonymous Class')]

,Bugs,commit_hash,Name,Kind,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,...,MaxEssential,MaxInheritanceTree,MaxNesting,PercentLackOfCohesion,PercentLackOfCohesionModified,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
0,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.ApplicationTest,Public Class,1.0,1.0,1.0,1.0,3.0,0.0,...,1.0,2.0,0,0.0,0.0,0.75,1,1,1,1
1,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MANAndroidDemo,Public Class,3.0,3.0,3.0,1.0,60.0,4.0,...,1.0,1.0,3,0.0,0.0,0.89,6,6,6,2
2,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MainActivity,Public Class,1.0,1.0,1.0,1.0,17.0,1.0,...,1.0,2.0,1,0.0,0.0,0.11,4,4,4,3
3,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.AboutUsActivity,Public Class,1.0,1.0,1.0,1.0,6.0,0.0,...,1.0,2.0,1,0.0,0.0,0.17,4,4,4,3
6,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.HttpDnsTLSSniSock...,Public Class,1.0,1.0,1.0,1.0,7.0,0.0,...,1.0,2.0,2,85.0,85.0,0.13,14,14,14,9
7,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity,Public Class,2.0,1.0,2.0,1.0,20.0,0.0,...,1.0,2.0,2,89.0,78.0,0.24,36,29,38,16
15,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.WebviewActivity,Public Class,3.0,3.0,4.0,2.0,32.0,1.0,...,8.0,2.0,3,78.0,78.0,0.19,26,26,30,19
16,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.WebviewActivity.W...,Class,1.0,1.0,1.0,1.0,7.0,0.0,...,1.0,2.0,2,85.0,85.0,0.09,14,14,14,9
20,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_api_demo.HttpDNS,Public Class,2.0,2.0,2.0,1.0,6.0,0.0,...,6.0,1.0,3,76.0,73.0,0.19,12,12,14,11
21,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_api_demo.HttpDNS.HostObject,Class,1.0,1.0,1.0,1.0,3.0,0.0,...,1.0,1.0,0,67.0,65.0,0.00,10,10,10,10


In [107]:
df_pr

,Bugs,commit_hash,Name,Kind,AvgCyclomatic,AvgCyclomaticModified,AvgCyclomaticStrict,AvgEssential,AvgLine,AvgLineBlank,...,MaxEssential,MaxInheritanceTree,MaxNesting,PercentLackOfCohesion,PercentLackOfCohesionModified,RatioCommentToCode,SumCyclomatic,SumCyclomaticModified,SumCyclomaticStrict,SumEssential
0,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.ApplicationTest,Public Class,1.0,1.0,1.0,1.0,3.0,0.0,...,1.0,2.0,0,0.0,0.0,0.75,1,1,1,1
1,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MANAndroidDemo,Public Class,3.0,3.0,3.0,1.0,60.0,4.0,...,1.0,1.0,3,0.0,0.0,0.89,6,6,6,2
2,1,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.man_android_demo.MainActivity,Public Class,1.0,1.0,1.0,1.0,17.0,1.0,...,1.0,2.0,1,0.0,0.0,0.11,4,4,4,3
3,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.AboutUsActivity,Public Class,1.0,1.0,1.0,1.0,6.0,0.0,...,1.0,2.0,1,0.0,0.0,0.17,4,4,4,3
6,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.HttpDnsTLSSniSock...,Public Class,1.0,1.0,1.0,1.0,7.0,0.0,...,1.0,2.0,2,85.0,85.0,0.13,14,14,14,9
7,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity,Public Class,2.0,1.0,2.0,1.0,20.0,0.0,...,1.0,2.0,2,89.0,78.0,0.24,36,29,38,16
8,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.http...,Anonymous Class,4.0,4.0,4.0,1.0,53.0,1.0,...,1.0,2.0,1,0.0,0.0,0.35,4,4,4,1
9,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.http...,Anonymous Class,2.0,2.0,2.0,1.0,7.0,0.0,...,1.0,2.0,1,0.0,0.0,1.22,2,2,2,1
10,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.init...,Anonymous Class,2.0,2.0,2.0,1.0,7.0,0.0,...,1.0,2.0,1,0.0,0.0,0.00,2,2,2,1
11,0,1e3e80823c5ed44d55c81a597e51bc556f348a74,alibaba.httpdns_android_demo.MainActivity.norm...,Anonymous Class,4.0,4.0,4.0,1.0,34.0,3.0,...,1.0,2.0,2,0.0,0.0,0.14,4,4,4,1
